
# Session 4 — Agentic AI (OpenAI): From Retrieval to Action

This lab upgrades the agent from a local model to **OpenAI**.  
You will connect an LLM to three tools and watch it *think → choose a tool → act → answer*.

**Tools available**
- `campus_retriever`: searches your existing RAG index (no rebuild)
- `calculator`: safe math
- `weather_live`: real weather via the free wttr.in API

> Keep your `data/` folder from Session 3 (must contain `chunks.json` and `embeddings.npy`).



## 0) Setup (run once)
Install agent + OpenAI bindings.


In [1]:
# Force Transformers to use PyTorch backend only
import os
os.environ['TRANSFORMERS_NO_TF'] = '1'
os.environ['USE_TF'] = '0'
print("Set TRANSFORMERS_NO_TF and USE_TF")


Set TRANSFORMERS_NO_TF and USE_TF


In [2]:

!pip -q install langchain langchain-openai sentence-transformers faiss-cpu transformers requests tiktoken



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\varsh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



## 1) Configure your OpenAI API key
Paste your key below (it only lives in this runtime). If already set in environment, this will detect it.


In [4]:

import os
key = os.getenv("OPENAI_API_KEY", None)
if not key:
    try:
        from getpass import getpass
        key = getpass("Enter your OPENAI_API_KEY (input hidden): ")
    except Exception:
        key = input("Enter your OPENAI_API_KEY: ")
os.environ["OPENAI_API_KEY"] = (key or "").strip()
print("OpenAI key configured." if key else "⚠️ No key detected.")


OpenAI key configured.



## 2) Load the existing RAG knowledge base
We reuse the embeddings and chunks from Lab 3 — no re-chunking and no re-embedding.


In [5]:

from sentence_transformers import SentenceTransformer
import json, numpy as np, faiss, os

EMBED_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

assert os.path.exists("data/chunks.json"), "Missing data/chunks.json (run Session 3 first)."
assert os.path.exists("data/embeddings.npy"), "Missing data/embeddings.npy (run Session 3 first)."

embedder = SentenceTransformer(EMBED_MODEL_NAME)
chunks = json.load(open("data/chunks.json","r",encoding="utf-8"))
emb = np.load("data/embeddings.npy")

index = faiss.IndexFlatIP(emb.shape[1])
index.add(emb)

print(f"Loaded {len(chunks)} chunks, embedding dim={emb.shape[1]}")


C:\Users\varsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded 2 chunks, embedding dim=384



## 3) Define the tools
- Retriever: semantic search over your campus guide  
- Calculator: safe eval with `sqrt` allowed  
- Weather (live): free endpoint from wttr.in (no key needed)


In [6]:

from langchain.tools import tool
import requests
from math import sqrt

@tool("campus_retriever")
def campus_retriever(query: str) -> str:
    """Retrieve relevant campus info from the indexed knowledge base."""
    qv = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    scores, idxs = index.search(qv, 4)
    ctx = []
    for i in idxs[0]:
        if int(i) < 0: 
            continue
        ctx.append(chunks[int(i)]['text'])
    return "\n\n".join(ctx)

@tool("calculator")
def calculator(expression: str) -> str:
    """Safely evaluate simple math (use +,-,*,/, sqrt())."""
    try:
        out = eval(expression, {"sqrt": sqrt, "__builtins__": {}})
        return str(out)
    except Exception as e:
        return f"Error: {e}"

@tool("weather_live")
def weather_live(city: str) -> str:
    """Fetch live weather using wttr.in (no API key). Return one-line summary."""
    try:
        r = requests.get(f"https://wttr.in/{city}?format=3", timeout=6)
        return r.text.strip()
    except Exception as e:
        return f"Error fetching weather: {e}"



## 4) Initialize the OpenAI LLM
We use `gpt-4o-mini` with low temperature for grounded answers.


In [7]:

from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
tools = [campus_retriever, calculator, weather_live]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # ReAct: think -> act -> observe
    verbose=True
)

print("Agent ready (OpenAI + tools).")


Agent ready (OpenAI + tools).


C:\Users\varsh\AppData\Local\Temp\ipykernel_4804\1128722206.py:7: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(



## 5) Demo — watch the agent think and act
You will see the ReAct trace in the output: Thought → Action → Observation → Answer.


In [9]:

tests = [
    "What are the library hours?",
    "What is 15 squared plus 10?",
    "What's the weather like in Bangalore?",
    "Find the sports complex hours and also tell me if it's likely to be hot in Chennai today."
]

for q in tests:
    print("\n🧠 Query:", q)
    print("-"*72)
    ans = agent.run(q)
    print("🤖 Final Answer:", ans)
    print("-"*72)



🧠 Query: What are the library hours?
------------------------------------------------------------------------


> Entering new AgentExecutor chain...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


## 6) Mini chat loop (optional)
Type your own questions. Type `exit` to stop.


In [ ]:

while True:
    try:
        q = input("You: ")
    except EOFError:
        break
    if q.strip().lower() in {"exit", "quit"}:
        print("Bye!")
        break
    try:
        print("Agent:", agent.run(q))
    except Exception as e:
        print("Agent error:", e)



## 7) Teaching notes
- OpenAI is a drop-in replacement that improves reasoning quality for agent demos.
- Temperature 0.2–0.4 is ideal for factual tasks; increase for creative writing.
- We reused the exact Session 3 index to show modular design.
- Discuss safety: tool limits, API quotas, prompt logging, privacy.
